## 项目背景

此项目通过对即将到期的用户历史行为数据进行分析,预测是否续费.

## 项目结果
最终对验证集的AUC为0.985185,对测试集的AUC为0.86

远高于之前使用的模型0.4152的准确率

模型有轻微的过拟合,主要原因是测试集的选择集中在某一天的流失用户.

## 数据清洗

原始数据提供

['event', 'distinct_id', 'date', 'time', 'entrance_source', 'vipflag','use_end', 'page_title', 'tab_name', '$title', 'pop_up_name','button_name', 'is_success', 'fail_reason', 'primary_class','date_start', 'date_end', 'module_name', 'recent_7_days_orders','payment_frequency', 'paid_tag']
 20个字段,通过补充用户信息,行为信息间组合变换等方法拓展到1500个字段.

训练集和验证集为同一天到期的用户
测试集为其他不同日期到期的用户

产品为订阅制的刚需电商工具类产品.
模型为基于随机森林的二分类模型.

行为数据为到期前四个月到到期前一个月的行为数据
tag为预测目标.即到期前一个月到到期后十五天内用户是否会续费,1为续费,0为不续费.


In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import  metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
import joblib

In [2]:
# data = pd.read_csv('./../proceedings_data/features_data_set.csv',encoding='utf_8_sig',index_col=0)


data = pd.read_csv('./../proceedings_data/12_29_features_data_set.csv',encoding='utf_8_sig',index_col=None)
user_info = pd.read_csv('./../source_data/ML_用户账号信息.csv',encoding='utf_8_sig',index_col=None)
# user_info = pd.read_csv('./../proceedings_data/user_info.csv',encoding='utf_8_sig',index_col=None)
data = pd.merge(data,user_info,how='left',left_on='distinct_id',right_on='second_id')
data.fillna(-1,inplace=True)
data

,distinct_id,date_<lambda_0>,date_count,entrance_source_<lambda>,vipflag_max,is_success_1_sum,is_success_0_sum,is_success_-1_sum,fail_reason_<lambda>,recent_7_days_orders_max,...,equipment_id,is_continuity_delivery,is_unilateralism,user_id_all,platform_type,num_shop,sellernick_all,utm_source_from_trade,is_stamps_autodeliver_open,average_dd
0,0乐豆宝贝0:乐乐,121,23875,list,1.0,0,117,23758,"[-1, '订单关闭失败', '[object Object]', '请选择您要操作的订单'...",3183.0,...,0BEB7873,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,千牛,0.0,71.000
1,13425511312x,119,14690,list,1.0,0,64,14626,"[-1, '订单关闭失败', '[object Object]', '请选择您要操作的订单'...",79.0,...,40BC1949,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,千牛,0.0,0.857
2,13625249156dn86,98,2735,index,1.0,311,41,2383,"[-1, '订单关闭失败', '[object Object]', '请选择您要操作的订单'...",13.0,...,94374c5dbb926bba31e5eb3e0fdbbc78,0.0,1.0,-1.0,-1.0,-1.0,-1.0,千牛,1.0,1.000
3,a330705168,120,19232,list,1.0,52,122,19058,"[-1, '订单关闭失败', '[object Object]', '请选择您要操作的订单'...",89.0,...,BE541CC6,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,千牛,0.0,4.571
4,ali_爱丽,111,5004,list,1.0,9,9,4986,"[-1, '订单关闭失败', '[object Object]', '请选择您要操作的订单'...",66.0,...,C16CEADB,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,千牛,0.0,0.142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,香奈尔要嫁人:华华,96,2774,index,1.0,0,4,2770,"[-1, '订单关闭失败', '[object Object]', '请选择您要操作的订单'...",1481.0,...,bbc349d896b541c4908f2211dd9dd5ae,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,0.0,38.571
184,馥居家居旗舰店:素若,104,4503,index,1.0,383,31,4089,"[-1, '订单关闭失败', '[object Object]', '请选择您要操作的订单'...",139.0,...,627ead4727b6edb3e31cf2bd8a26e348,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,2.142
185,驰昌数码批发,118,3000,detail,1.0,0,17,2983,"[-1, '订单关闭失败', '[object Object]', '请选择您要操作的订单'...",118.0,...,242A0E0B,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,千牛,0.0,1.285
186,黄莉娇76779931,121,31237,list,1.0,0,43,31194,"[-1, '订单关闭失败', '[object Object]', '请选择您要操作的订单'...",419.0,...,3BD2EFAC,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,千牛,0.0,3.571


In [3]:
features = list(data.columns)
try :
    features.remove('distinct_id')
    features.remove('paid_tag_max')
except:
    print('无"distinct_id"或"paid_tag_max"字段')
obj_features = list(set(data.select_dtypes(include='object').columns)&set(features))
tag = 'paid_tag_max'

In [4]:
for col in obj_features:
    values = data[col].unique()
    for index_ in range(len(values)):
        data[col] = data[col].apply(lambda x:index_ if x==values[index_] else x)

for col in ['vipflag_max','recent_7_days_orders_max','payment_frequency_max']:
    data[col].fillna(0,inplace=True)

In [5]:
x,y = data[features],data[tag]#选取列
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)#划分数据集
train= np.column_stack((x_train,y_train))
test = np.column_stack((x_test, y_test))

In [6]:
##默认参

rf0 = RandomForestClassifier(oob_score=True, random_state=0)
# rf0.fit(x_train,y_train.astype('int'))
rf0.fit(x_train,y_train)
print (rf0.oob_score_)
y_predprob = rf0.predict_proba(x_test)[:,1]
print( "AUC Score (Train): %f" % metrics.roc_auc_score(y_test, y_predprob))
#拟合效果rf0.oob_score_=0.8015267175572519
rf0.get_params()

0.8015267175572519
AUC Score (Train): 0.960494


{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': True,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}

In [7]:

test = pd.concat([x_test,y_test],axis=1)
test.reset_index(drop=True,inplace=True)
# test.drop(columns='Unnamed: 0',inplace=True)
test = pd.concat([test,pd.DataFrame(y_predprob)],axis=1)


result = test[['paid_tag_max',0]]
result['predict_tag'] = result[0].apply(lambda x:1 if x >0.5 else 0)
result.columns = ['truth_tag','probability_tag','predict_tag']
result = result[['truth_tag','predict_tag','probability_tag']]
wrong_result = result[result['truth_tag']!=result['predict_tag']]
display(wrong_result)
print(result.shape[0],wrong_result.shape[0])
print(1-wrong_result.shape[0]/result.shape[0])


joblib.dump(rf0,'original_RF.pkl')
result

C:\Users\Xuzipei\AppData\Local\Temp\ipykernel_21700\1048075774.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['predict_tag'] = result[0].apply(lambda x:1 if x >0.5 else 0)


,truth_tag,predict_tag,probability_tag
8,0.0,1,0.60
9,0.0,1,0.59
14,0.0,1,0.58
21,1.0,0,0.41
29,0.0,1,0.83
50,0.0,1,0.57


57 6
0.8947368421052632


,truth_tag,predict_tag,probability_tag
0,1.0,1,0.62
1,0.0,0,0.10
2,1.0,1,0.69
3,0.0,0,0.32
4,0.0,0,0.15
5,0.0,0,0.23
6,1.0,1,0.90
7,0.0,0,0.35
8,0.0,1,0.60
9,0.0,1,0.59


In [8]:
n_estimators = np.arange(100, 3000, step=100)
max_features = ["auto", "sqrt", "log2"]
max_depth = list(np.arange(10, 200, step=5)) + [None]
min_samples_split = np.arange(2, 10, step=2)
min_samples_leaf = [1, 2,3, 4]
bootstrap = [True, False]

param_grid = {
    "n_estimators": n_estimators,
    "max_features": max_features,
    "max_depth": max_depth,
    "min_samples_split": min_samples_split,
    "min_samples_leaf": min_samples_leaf,
    "bootstrap": bootstrap,
}


rf1 = RandomForestClassifier(oob_score=True)
# random_cv = RandomizedSearchCV(
#     rf1, param_grid, n_iter=100, cv=5, scoring="roc_auc", n_jobs=-1)

random_cv = RandomizedSearchCV(
    rf1, param_grid, n_iter=100, cv=5, scoring="roc_auc", n_jobs=-1,random_state=2)

random_cv.fit(x_train,y_train)
# print (random_cv.oob_score_)
y_pred = random_cv.predict_proba(x_test)[:,1]
print( "AUC Score (Train): %f" % metrics.roc_auc_score(y_test, y_pred))
#拟合效果rf0.oob_score_=0.8778625954198473
print(random_cv.best_score_)
print(random_cv.best_params_)

test = pd.concat([x_test,y_test],axis=1)
test.reset_index(drop=True,inplace=True)
# test.drop(columns='Unnamed: 0',inplace=True)
test = pd.concat([test,pd.DataFrame(y_pred)],axis=1)

result = test[['paid_tag_max',0]]
result['predict_tag'] = result[0].apply(lambda x:1 if x >0.5 else 0)
result.columns = ['truth_tag','probability_tag','predict_tag']
result = result[['truth_tag','predict_tag','probability_tag']]
wrong_result = result[result['truth_tag']!=result['predict_tag']]
display(wrong_result)
print(result.shape[0],wrong_result.shape[0])
print(1-wrong_result.shape[0]/result.shape[0])

joblib.dump(random_cv,'random_cv_RF.pkl')

print(result)

D:\workspace\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
215 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
215 fits failed with the following error:
Traceback (most recent call last):
  File "D:\workspace\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\workspace\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 437, in fit
    raise ValueError("Out of bag estimation only available if bootstrap=True")
ValueError: Out of bag estimation only available if bootstrap=True

  warnings.warn(some_fits_failed_message, FitFailedWarning)
D:\works

AUC Score (Train): 0.951852
0.9032051282051283
{'n_estimators': 1600, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 125, 'bootstrap': True}


C:\Users\Xuzipei\AppData\Local\Temp\ipykernel_21700\3466314148.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['predict_tag'] = result[0].apply(lambda x:1 if x >0.5 else 0)


,truth_tag,predict_tag,probability_tag
8,0.0,1,0.554375
9,0.0,1,0.583125
14,0.0,1,0.563125
18,0.0,1,0.589375
21,1.0,0,0.347500
29,0.0,1,0.835625
50,0.0,1,0.591875


57 7
0.8771929824561404
    truth_tag  predict_tag  probability_tag
0         1.0            1         0.563125
1         0.0            0         0.080000
2         1.0            1         0.734375
3         0.0            0         0.287500
4         0.0            0         0.132500
5         0.0            0         0.235000
6         1.0            1         0.878125
7         0.0            0         0.303750
8         0.0            1         0.554375
9         0.0            1         0.583125
10        0.0            0         0.066875
11        1.0            1         0.907500
12        1.0            1         0.768125
13        0.0            0         0.351250
14        0.0            1         0.563125
15        1.0            1         0.735000
16        0.0            0         0.282500
17        0.0            0         0.355000
18        0.0            1         0.589375
19        0.0            0         0.255625
20        1.0            1         0.804375
21      

In [9]:
# from sklearn.model_selection import GridSearchCV
# grid_params = {
#     "n_estimators": [i for i in range(1400,1801,10)],
#     "max_features": ['auto','sqrt'],
#     "max_depth": [i for i in range(10,121,10)],
#     "min_samples_split": [1,2,3,4,5,6,7,8,9],
#     "min_samples_leaf": [1, 2,3],
#     "bootstrap": [True],
# }
#
#
#
# rf2 = RandomForestClassifier()
# grid_cv = GridSearchCV(rf2,grid_params,n_jobs=-1)
# grid_cv.fit(x_train,y_train)
# y_pred_grid = grid_cv.predict_proba(x_test)[:,1]
# print( "AUC Score (Train): %f" % metrics.roc_auc_score(y_test, y_pred_grid))
# #拟合效果rf0.oob_score_=0.8778625954198473
# print(grid_cv.best_score_)
# print(grid_cv.best_params_)
# joblib.dump(random_cv,'grid_cv_RF.pkl')
#


In [10]:
from scipy.stats import pearsonr

In [19]:
grid_cv_best = joblib.load('grid_cv_RF.pkl')


y_pred_grid = grid_cv_best.predict_proba(x_test)[:,1]

test = pd.concat([x_test,y_test],axis=1)
test.reset_index(drop=True,inplace=True)
# test.drop(columns='Unnamed: 0',inplace=True)
test = pd.concat([test,pd.DataFrame(y_pred_grid)],axis=1)

print( "AUC Score (Train): %f" % metrics.roc_auc_score(y_test, y_pred_grid))

result = test[['paid_tag_max',0]]
result['predict_tag'] = result[0].apply(lambda x:1 if x >0.5 else 0)
result.columns = ['truth_tag','probability_tag','predict_tag']
result = result[['truth_tag','predict_tag','probability_tag']]
wrong_result = result[result['truth_tag']!=result['predict_tag']]
display(wrong_result)
print(result.shape[0],wrong_result.shape[0])
print(1-wrong_result.shape[0]/result.shape[0])

result


AUC Score (Train): 0.985185


C:\Users\Xuzipei\AppData\Local\Temp\ipykernel_21700\563131604.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['predict_tag'] = result[0].apply(lambda x:1 if x >0.5 else 0)


,truth_tag,predict_tag,probability_tag
9,0.0,1,0.653913
18,0.0,1,0.560870
29,0.0,1,0.841304


57 3
0.9473684210526316


,truth_tag,predict_tag,probability_tag
0,1.0,1,0.846957
1,0.0,0,0.050000
2,1.0,1,0.921304
3,0.0,0,0.106522
4,0.0,0,0.071304
5,0.0,0,0.070435
6,1.0,1,0.953478
7,0.0,0,0.112609
8,0.0,0,0.190870
9,0.0,1,0.653913


In [12]:
features_importance = pd.concat([pd.DataFrame(test.columns),pd.DataFrame(rf0.feature_importances_)],axis=1)
features_importance.columns = ['features','importances']
features_importance['importances']= features_importance['importances'].apply(lambda x:x*100)
features_importance.sort_values(by='importances',ascending=False)

,features,importances
37,event_YY_PurchaseResultsDetails,5.128502
1493,is_main,2.415599
57,event_UserLabel,1.969016
560,pop_up_name_第一步,1.644005
13,is_success_0,1.517269
...,...,...
609,pop_up_name_阻止买家拍单,0.000000
608,pop_up_name_阻止此买家拍单,0.000000
767,button_name_开,0.000000
1534,paid_tag_max,NaN


In [13]:
data1 = pd.read_csv('./../proceedings_data/06_10_features_data_set.csv',encoding='utf_8_sig')
data2 = pd.read_csv('./../proceedings_data/07_15_features_data_set.csv',encoding='utf_8_sig')
data3 = pd.read_csv('./../proceedings_data/08_17_features_data_set.csv',encoding='utf_8_sig')
data4 = pd.read_csv('./../proceedings_data/09_25_features_data_set.csv',encoding='utf_8_sig')
data5 = pd.read_csv('./../proceedings_data/10_30_features_data_set.csv',encoding='utf_8_sig')

verification_data = pd.concat([data1,data2,data3,data4,data5])
verification_data = pd.merge(verification_data,user_info,how='left',left_on='distinct_id',right_on='second_id')
verification_data.reset_index(inplace=True,drop=True)
verification_data.fillna(-1,inplace=True)
verification_data

,distinct_id,date_<lambda_0>,date_count,entrance_source_<lambda>,vipflag_max,is_success_1_sum,is_success_0_sum,is_success_-1_sum,fail_reason_<lambda>,recent_7_days_orders_max,...,equipment_id,is_continuity_delivery,is_unilateralism,user_id_all,platform_type,num_shop,sellernick_all,utm_source_from_trade,is_stamps_autodeliver_open,average_dd
0,fpaija,61,746,index,1.0,88,4,654,"[-1, '订单关闭失败', '[object Object]', '请选择您要操作的订单'...",5.0,...,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,0.428
1,huihui198311,71,1163,index,1.0,106,4,1053,"[-1, '订单关闭失败', '[object Object]', '请选择您要操作的订单'...",10.0,...,E73E2026,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,千牛,0.0,0.285
2,laoyang0410,135,17532,tradeList,1.0,2014,112,15406,"[-1, '订单关闭失败', '[object Object]', '请选择您要操作的订单'...",91.0,...,a1329be5dc1afcfc4e8cfa546223bf84,1.0,1.0,-1.0,-1.0,-1.0,-1.0,千牛,0.0,7.142
3,mengya08,19,39,index,1.0,0,0,39,"[-1, '订单关闭失败', '[object Object]', '请选择您要操作的订单'...",3764.0,...,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,224.714
4,snacksshine,69,4259,index,2.0,30,9,4220,"[-1, '订单关闭失败', '[object Object]', '请选择您要操作的订单'...",3.0,...,5daa20abf38695a50c7b83edf3e4eafa,0.0,0.0,-1.0,-1.0,-1.0,-1.0,千牛,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,网柚数码店:大大,2,11,index,1.0,0,1,10,"[-1, '订单关闭失败', '[object Object]', '请选择您要操作的订单'...",5071.0,...,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,633.714
137,花田鼠旗舰店,5,8,index,1.0,0,0,8,"[-1, '订单关闭失败', '[object Object]', '请选择您要操作的订单'...",763.0,...,3b0791366b1a067036a8f90fb06b4f05,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,43.571
138,蒲志明,121,11235,index,1.0,1131,91,10013,"[-1, '订单关闭失败', '[object Object]', '请选择您要操作的订单'...",56.0,...,F2FAF081,0.0,1.0,-1.0,-1.0,-1.0,-1.0,千牛,0.0,5.142
139,陌上花开富贵竹88,29,719,index,1.0,98,1,620,"[-1, '订单关闭失败', '[object Object]', '请选择您要操作的订单'...",41.0,...,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,0.000


In [14]:
verification_data['paid_tag_max'].value_counts()

0.0    100
1.0     41
Name: paid_tag_max, dtype: int64

In [21]:
features = list(verification_data.columns)
features.remove('distinct_id')
features.remove('paid_tag_max')
obj_features = list(set(verification_data.select_dtypes(include='object').columns)&set(features))
tag = 'paid_tag_max'

for col in obj_features:
    values = verification_data[col].unique()
    for index_ in range(len(values)):
        verification_data[col] = verification_data[col].apply(lambda x:index_ if x==values[index_] else x)

for col in ['vipflag_max','recent_7_days_orders_max','payment_frequency_max']:
    verification_data[col].fillna(0,inplace=True)

verification_x,verification_y = verification_data[features],verification_data[tag]#选取列
verification_y_pred_origin = rf0.predict_proba(verification_x)[:,1]
verification_result_origin =  pd.concat([pd.DataFrame(verification_data['paid_tag_max']),pd.DataFrame(verification_y_pred_origin)],axis=1)

print( "AUC Score (Train): %f" % metrics.roc_auc_score(verification_y, verification_y_pred_origin))

verification_result_origin['predict_tag'] = verification_result_origin[0].apply(lambda x:1 if x >0.5 else 0)
verification_result_origin.columns = ['truth_tag','probability_tag','predict_tag']
verification_result_origin = verification_result_origin[['truth_tag','predict_tag','probability_tag']]
wrong_verification_result_origin = verification_result_origin[verification_result_origin['truth_tag']!=verification_result_origin['predict_tag']]
display(wrong_verification_result_origin)
print(verification_result_origin.shape[0],wrong_verification_result_origin.shape[0])
print(1-wrong_verification_result_origin.shape[0]/verification_result_origin.shape[0])
display(verification_result_origin)
# result = test[['paid_tag_max',0]]
# result['predict_tag'] = result[0].apply(lambda x:1 if x >0.5 else 0)
# result.columns = ['truth_tag','probability_tag','predict_tag']
# result = result[['truth_tag','predict_tag','probability_tag']]
# wrong_result = result[result['truth_tag']!=result['predict_tag']]
# display(wrong_result)
# print(result.shape[0],wrong_result.shape[0])
# print(1-wrong_result.shape[0]/result.shape[0])

AUC Score (Train): 0.829390


,truth_tag,predict_tag,probability_tag
10,0.0,1,0.51
14,0.0,1,0.53
16,0.0,1,0.56
23,1.0,0,0.44
27,1.0,0,0.49
41,0.0,1,0.52
44,1.0,0,0.40
47,0.0,1,0.61
52,0.0,1,0.56
53,1.0,0,0.46


141 36
0.7446808510638299


,truth_tag,predict_tag,probability_tag
0,0.0,0,0.21
1,1.0,1,0.53
2,1.0,1,0.62
3,0.0,0,0.23
4,0.0,0,0.49
...,...,...,...
136,0.0,0,0.22
137,0.0,1,0.55
138,1.0,1,0.78
139,1.0,0,0.31


In [22]:
features = list(verification_data.columns)
features.remove('distinct_id')
features.remove('paid_tag_max')
obj_features = list(set(verification_data.select_dtypes(include='object').columns)&set(features))
tag = 'paid_tag_max'

for col in obj_features:
    values = verification_data[col].unique()
    for index_ in range(len(values)):
        verification_data[col] = verification_data[col].apply(lambda x:index_ if x==values[index_] else x)

for col in ['vipflag_max','recent_7_days_orders_max','payment_frequency_max']:
    verification_data[col].fillna(0,inplace=True)

verification_x,verification_y = verification_data[features],verification_data[tag]#选取列
verification_y_pred_random_cv = random_cv.predict_proba(verification_x)[:,1]
verification_result_random_cv =  pd.concat([pd.DataFrame(verification_data['paid_tag_max']),pd.DataFrame(verification_y_pred_random_cv)],axis=1)

print( "AUC Score (Train): %f" % metrics.roc_auc_score(verification_y, verification_y_pred_random_cv))

verification_result_random_cv['predict_tag'] = verification_result_random_cv[0].apply(lambda x:1 if x >0.5 else 0)
verification_result_random_cv.columns = ['truth_tag','probability_tag','predict_tag']
verification_result_random_cv = verification_result_random_cv[['truth_tag','predict_tag','probability_tag']]
wrong_verification_result_random_cv = verification_result_random_cv[verification_result_random_cv['truth_tag']!=verification_result_random_cv['predict_tag']]
display(wrong_verification_result_random_cv)
print(verification_result_random_cv.shape[0],wrong_verification_result_random_cv.shape[0])
print(1-wrong_verification_result_random_cv.shape[0]/verification_result_random_cv.shape[0])
display(verification_result_random_cv)
# result = test[['paid_tag_max',0]]
# result['predict_tag'] = result[0].apply(lambda x:1 if x >0.5 else 0)
# result.columns = ['truth_tag','probability_tag','predict_tag']
# result = result[['truth_tag','predict_tag','probability_tag']]
# wrong_result = result[result['truth_tag']!=result['predict_tag']]
# display(wrong_result)
# print(result.shape[0],wrong_result.shape[0])
# print(1-wrong_result.shape[0]/result.shape[0])

AUC Score (Train): 0.853537


,truth_tag,predict_tag,probability_tag
7,1.0,0,0.478125
16,0.0,1,0.577500
23,1.0,0,0.472500
27,1.0,0,0.465625
33,0.0,1,0.511250
40,1.0,0,0.488750
41,0.0,1,0.546250
44,1.0,0,0.401875
47,0.0,1,0.576875
52,0.0,1,0.545625


141 35
0.75177304964539


,truth_tag,predict_tag,probability_tag
0,0.0,0,0.219375
1,1.0,1,0.539375
2,1.0,1,0.619375
3,0.0,0,0.221875
4,0.0,0,0.439375
...,...,...,...
136,0.0,0,0.177500
137,0.0,1,0.531250
138,1.0,1,0.751250
139,1.0,0,0.286250


In [23]:
features = list(verification_data.columns)
features.remove('distinct_id')
features.remove('paid_tag_max')
obj_features = list(set(verification_data.select_dtypes(include='object').columns)&set(features))
tag = 'paid_tag_max'

for col in obj_features:
    values = verification_data[col].unique()
    for index_ in range(len(values)):
        verification_data[col] = verification_data[col].apply(lambda x:index_ if x==values[index_] else x)

for col in ['vipflag_max','recent_7_days_orders_max','payment_frequency_max']:
    verification_data[col].fillna(0,inplace=True)

verification_x,verification_y = verification_data[features],verification_data[tag]#选取列
verification_y_pred_grid_cv = grid_cv_best.predict_proba(verification_x)[:,1]
verification_result_grid_cv =  pd.concat([pd.DataFrame(verification_data['paid_tag_max']),pd.DataFrame(verification_y_pred_grid_cv)],axis=1)

print( "AUC Score (Train): %f" % metrics.roc_auc_score(verification_y, verification_y_pred_grid_cv))

verification_result_grid_cv['predict_tag'] = verification_result_grid_cv[0].apply(lambda x:1 if x >0.5 else 0)
verification_result_grid_cv.columns = ['truth_tag','probability_tag','predict_tag']
verification_result_grid_cv = verification_result_grid_cv[['truth_tag','predict_tag','probability_tag']]
wrong_verification_result_grid_cv = verification_result_grid_cv[verification_result_grid_cv['truth_tag']!=verification_result_grid_cv['predict_tag']]
display(wrong_verification_result_grid_cv)
print(verification_result_grid_cv.shape[0],wrong_verification_result_grid_cv.shape[0])
print(1-wrong_verification_result_grid_cv.shape[0]/verification_result_grid_cv.shape[0])
display(verification_result_grid_cv)
# result = test[['paid_tag_max',0]]
# result['predict_tag'] = result[0].apply(lambda x:1 if x >0.5 else 0)
# result.columns = ['truth_tag','probability_tag','predict_tag']
# result = result[['truth_tag','predict_tag','probability_tag']]
# wrong_result = result[result['truth_tag']!=result['predict_tag']]
# display(wrong_result)
# print(result.shape[0],wrong_result.shape[0])
# print(1-wrong_result.shape[0]/result.shape[0])

AUC Score (Train): 0.860000


,truth_tag,predict_tag,probability_tag
7,1.0,0,0.493913
10,0.0,1,0.506957
16,0.0,1,0.600000
17,0.0,1,0.522174
33,0.0,1,0.551304
41,0.0,1,0.573043
44,1.0,0,0.453478
47,0.0,1,0.625217
52,0.0,1,0.607826
54,1.0,0,0.349130


141 30
0.7872340425531915


,truth_tag,predict_tag,probability_tag
0,0.0,0,0.223043
1,1.0,1,0.618696
2,1.0,1,0.696957
3,0.0,0,0.234348
4,0.0,0,0.482609
...,...,...,...
136,0.0,0,0.136522
137,0.0,0,0.464783
138,1.0,1,0.809565
139,1.0,0,0.337391


In [ ]:
def judge_whether_prediction_is_more_accurate(truth_tag,origin_probability,probability_after_optimization):
    if truth_tag<origin_probability<probability_after_optimization:
        return 0
    elif probability_after_optimization<origin_probability<truth_tag:
        return 0
    else:
        return 1

In [ ]:
result_o_and_r = pd.concat([verification_result_origin,verification_result_random_cv],join='outer',axis=1)
result_o_and_r.columns = ['truth_tag','o_pre_tag','o_probability','truth_tag2','random__pre_tag','random_probability']
result_o_and_r.drop(columns='truth_tag2',inplace=True)
result_o_and_r['optimization_result'] = result_o_and_r.apply(lambda df:judge_whether_prediction_is_more_accurate(truth_tag=df['truth_tag'],
                                                                                                                 origin_probability=df['o_probability'],
                                                                                                                 probability_after_optimization=df['random_probability']),axis=1)
display(result_o_and_r)
result_o_and_r['optimization_result'].value_counts()

In [ ]:
features = list(verification_data.columns)
features.remove('distinct_id')
features.remove('paid_tag_max')
obj_features = list(set(verification_data.select_dtypes(include='object').columns)&set(features))
tag = 'paid_tag_max'

for col in obj_features:
    values = verification_data[col].unique()
    for index_ in range(len(values)):
        verification_data[col] = verification_data[col].apply(lambda x:index_ if x==values[index_] else x)

for col in ['vipflag_max','recent_7_days_orders_max','payment_frequency_max']:
    verification_data[col].fillna(0,inplace=True)

verification_x,verification_y = verification_data[features],verification_data[tag]#选取列
verification_y_pred_grid_cv = grid_cv_best.predict_proba(verification_x)[:,1]
verification_result_grid_cv =  pd.concat([pd.DataFrame(verification_data['paid_tag_max']),pd.DataFrame(verification_y_pred_grid_cv)],axis=1)

verification_result_grid_cv['predict_tag'] = verification_result_grid_cv[0].apply(lambda x:1 if x >0.5 else 0)
verification_result_grid_cv.columns = ['truth_tag','probability_tag','predict_tag']
verification_result_grid_cv = verification_result_grid_cv[['truth_tag','predict_tag','probability_tag']]
wrong_verification_result_grid_cv = verification_result_grid_cv[verification_result_grid_cv['truth_tag']!=verification_result_grid_cv['predict_tag']]
display(wrong_verification_result_grid_cv)
print(verification_result_grid_cv.shape[0],wrong_verification_result_grid_cv.shape[0])
print(1-wrong_verification_result_grid_cv.shape[0]/verification_result_grid_cv.shape[0])
display(verification_result_grid_cv)
# result = test[['paid_tag_max',0]]
# result['predict_tag'] = result[0].apply(lambda x:1 if x >0.5 else 0)
# result.columns = ['truth_tag','probability_tag','predict_tag']
# result = result[['truth_tag','predict_tag','probability_tag']]
# wrong_result = result[result['truth_tag']!=result['predict_tag']]
# display(wrong_result)
# print(result.shape[0],wrong_result.shape[0])
# print(1-wrong_result.shape[0]/result.shape[0])